### Feature engineering

- Remove unnecessary columns to reduce dataframe size
- Filter train records according to min/max langtitude and longitude in grid info
- Generate grid info and aggregate records accroding to VIN and grid id

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import timeit
%matplotlib inline

# loading training files
ecar_training_data_path = 'data/training/ecar/'
rcar_training_data_path = 'data/training/rcar/'

ecar_filenames = [f for f in os.listdir(ecar_training_data_path)]
rcar_filenames = [f for f in os.listdir(rcar_training_data_path)]

# define drop columns
ecar_drop_columns = ['work_mode','mileage','avg_fuel_consumption','system_mode']
rcar_drop_columns = ['power_mode','mileage','fuel_consumption']
common_drop_columns = ['lat','lon','date_time']

# loading grid min/max lantitude and longitude
grids = pd.read_csv('data/grid_info.csv')
grid_max_lantitude = grids['latitude_to'].max()
grid_min_lantitude = grids['latitude_from'].min()
grid_max_longitude = grids['longitude_to'].max()
grid_min_longitude = grids['longitude_from'].min()

In [85]:
def round_down_lat_lon(df):
    df['lat'] = df.lat.round(3)
    df['lon'] = df.lon.round(3)
    return df
        
def filter_data_by_lan_lon(df):
    df = df[(df.lat >= grid_min_lantitude ) & (df.lat <= grid_max_lantitude)]
    df = df[(df.lon >= grid_min_longitude ) & (df.lon <= grid_max_longitude)]
    return df

def add_grid_id(df):
    df_with_grid_id = pd.DataFrame(index=[0], columns=['car_id', 'date_time','speed','lat','lon','grid_id'])
    df_with_grid_id = df_with_grid_id.fillna(-1)
    for i in range(len(grids)):
        min_lan, max_lan = grids['latitude_from'][i], grids['latitude_to'][i]
        min_lon, max_lon, grid_id = grids['longitude_from'][i], grids['longitude_to'][i], grids['grid_id'][i]
        df_ = df[(df.lat >= min_lan ) & (df.lat <= max_lan)]
        df_ = df_[(df_.lon >= min_lon ) & (df_.lon <= max_lon)]
        df_['grid_id'] = grid_id
        df_with_grid_id = df_with_grid_id.append(df_)
    return df_with_grid_id

def format_date(date_time):
    return date_time[:-9].replace('-','')

def retrieve_hour(date_time):
    return date_time[-8:-6]

def preprocess_data(dataType='rcar'):
    # define common variables
    filenames = rcar_filenames
    filepath = rcar_training_data_path
    drop_columns = rcar_drop_columns
    
    if dataType == 'ecar':
        filenames = ecar_filenames
        filepath = ecar_training_data_path
        drop_columns = ecar_drop_columns
    
    df_ = pd.DataFrame(index=[0], columns=['car_id', 'date','hour','speed', 'grid_id'])
    df_ = df_.fillna(-1)
    for filename in filenames:
        # if is directory skip
        if os.path.isdir(filepath + filename):
            continue
        # if file is already generated skip
        if os.path.exists(filepath + 'processed/' + filename):
            continue
        # init run time
        start = timeit.default_timer()
        # loading data
        df = pd.read_csv(filepath + filename)
        # drop uncessary columns
        df = df.drop(columns=drop_columns)
        # filter data by lat and lon
        df = filter_data_by_lan_lon(df)
        # format data_time column
        df['hour'] = df['date_time'].apply(lambda x : retrieve_hour(x))
        df['date'] = df['date_time'].apply(lambda x : format_date(x))
        # generate new column grid_id
        df = add_grid_id(df)
        # filter data by grid_id
        df = df[df.grid_id != -1]
        # drop lat and lon
        df = df.drop(columns=common_drop_columns)
        # appending df
        df_ = df_.append(df)
        # remove grid_id = -1
        df_ = df_[df_.grid_id != -1]
        # group by below column and get average speed for each record
        df_ = df.groupby(['car_id', 'date','hour','grid_id'], as_index=False).mean()
        # round average speed
        df_.speed = df_.speed.round(3)
        # write final file
        df_.to_csv(filepath + 'processed/' + filename)
        # log run time
        print("Finish preprocess file:[", filename, "] total cost:[", timeit.default_timer() - start ,"]")

In [87]:
preprocess_data('rcar')

In [86]:
preprocess_data('ecar')